# 4. 실시간 데이터 처리를 위한 최적화

* SparkSession을 생성하고 소켓으로 데이터를 수신하도록 스트리밍 설정을 하고 원하는 형태의 데이터만 다룰 수 있도록 합니다.
* 해당 데이터를 window와 watermarking을 다루는 방법을 익힙니다.

결과물 예시

| window                                    | word   | count |
|-------------------------------------------|--------|-------|
| {2024-11-09 11:14:50, 2024-11-09 11:15:00} | 치킨   | 1     |
| {2024-11-09 11:14:50, 2024-11-09 11:15:00} | 피자   | 2     |
| {2024-11-09 11:14:30, 2024-11-09 11:14:40} | 햄버거 | 1     |
| {2024-11-09 11:13:00, 2024-11-09 11:13:10} | 치킨   | 2     |
| {2024-11-09 11:13:40, 2024-11-09 11:13:50} | 피자   | 2     |
| {2024-11-09 11:13:40, 2024-11-09 11:13:50} | 햄버거 | 1     |



### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, current_timestamp, window

# SparkSession 생성
# SparkSession은 Spark 애플리케이션의 기본 진입점으로, Structured Streaming 기능을 사용하기 위해 필요합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = SparkSession.builder.appName("StructuredNetworkWordCountWithFixedWindow").getOrCreate()

In [ ]:
# 소켓 스트림 데이터 수신 설정
# localhost의 9999 포트에서 텍스트 데이터를 실시간으로 수신하여 스트림을 생성합니다.
# nc -lk 9999 명령어로 소켓 서버를 열면, 해당 터미널에 입력하는 텍스트가 Spark 스트리밍 애플리케이션으로 전달됩니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
lines = spark.____________.____________("socket").option("host", "localhost").option("port", 9999).load()

In [ ]:
# 현재 시간 열 추가
# 데이터가 수신된 시간을 기준으로 timestamp 열을 추가하여 이벤트 시간 기반 처리가 가능하게 합니다.
# 이렇게 추가된 타임스탬프는 워터마킹 및 윈도우 집계 시 기준이 됩니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.current_timestamp.html
lines_with_timestamp = lines.____________("timestamp", ____________())

In [ ]:
# 단어 분리
# 각 줄(line)을 공백(" ")으로 분리하여 개별 단어로 나눈 후 각 단어를 행(row)으로 만듭니다.
# explode(split(...))를 사용하여 각 텍스트 줄을 단어 단위로 분리하고, alias("word")로 열 이름을 지정합니다.
# 또한, timestamp 열을 유지하여 이후 집계에 사용됩니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html
words = lines_with_timestamp.select(____________(split(lines.value, " ")).alias("word"), "timestamp")


In [ ]:
# 워터마크 및 고정 윈도우 설정
# withWatermark("timestamp", "10 second")는 10초를 초과한 지연 데이터는 집계에서 제외하도록 설정합니다.
# window("timestamp", "10 second")는 10초 간격의 고정 윈도우를 설정하여 단어의 출현 빈도를 집계합니다.
# 이렇게 설정하면 10초마다 새로운 윈도우가 시작되며, 각 윈도우마다 독립적으로 단어 빈도가 집계됩니다.
# groupBy(...).count()는 각 윈도우와 단어별로 발생 빈도를 집계합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withWatermark.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html
wordCounts = words \
    .____________("timestamp", "10 second") \
    .groupBy(____________("timestamp", "10 second"), "word") \
    .count()

In [ ]:
# 스트리밍 결과 출력 설정
# outputMode("complete") - 모든 집계 결과를 출력합니다. 모든 단어의 빈도가 매번 전체적으로 출력됩니다.
# format("console") - 결과를 콘솔에 출력하도록 설정합니다.
# option("truncate", "false") - 출력 결과가 잘리지 않도록 설정하여 전체 데이터를 확인할 수 있게 합니다.
# .start()는 스트리밍 쿼리를 시작하고, awaitTermination()은 쿼리가 종료될 때까지 기다립니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.trigger.html
query = wordCounts.____________ \
    .____________("complete") \
    .____________("console") \
    .option("truncate", "false") \
    .start()

# 스트리밍 쿼리를 계속 실행하여 데이터 수신을 대기합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.StreamingQuery.awaitTermination.html
query.awaitTermination()
